In [ ]:
import random

from typing import Literal


def gen_name(
    minlen=1,
    maxlen=8,
    case: Literal["lower", "snake", "uppercamel", "upper"] = "lower",
):
    length = random.randint(minlen, maxlen)
    word = ""
    for i in range(length):
        c = chr(random.randint(97, 122))
        if case == "upper":
            c = c.upper()
        elif case == "uppercamel":
            if i == 0:
                c = c.upper()
            elif random.random() > 0.75:
                c = c.upper()

        word += c
    if case == "snake":
        s = random.randint(0, length)
        word = word[:s] + "_" + word[s:]

    return word


def randomize_names(tokens: list[str], tags: list[str]):
    """Randomize names of tokens with arbitrary names.

    Affected classes: `pa`, `mo`, `fnme`, `fnas`, `fnsa`, `va`, `at`
    """

    renameable = ["pa", "mo", "fnme", "fnas", "fnsa", "va", "at"]

    renamed = [False] * len(tokens)
    tokens_new = tokens.copy()
    for i, (token, tag) in enumerate(zip(tokens, tags)):
        print(renamed)
        if tag in renameable and not renamed[i]:
            newname = gen_name()
            for j in range(i, len(tokens)):
                if tokens[j] == token:
                    renamed[j] = True
                    tokens_new[j] = newname

    return tokens_new


randomize_names(
    ["x", "=", "f", "(", "x", ")"], ["va", "opas", "fnsa", "brop", "va", "brcl"]
)

In [ ]:
random.randint(0, 25)
ord("_")


In [ ]:
gen_name(case="snake")


## markov process


In [ ]:
transitions = {
    "[start]": {"va": 0.4, "kwio": 0.2, "fnsa": 0.2},
    "va": {"opas": 0.5},
    "opas": {"va": 0.5, "fnsa": 0.5},
    "fnsa": {"brop": 1.0},
    "kwio": {"va": 0.4, "fnsa": 0.3},
    "brop": {"va": 0.6, "brcl": 0.4},
}


def gen_markov(seq=["[start]"], maxlen=20):
    for i in range(maxlen):
        k = seq[-1]

        if k not in transitions.keys() or k == "[end]":
            break
        n = random.choices(
            population=list(transitions[k].keys()),
            weights=list(transitions[k].values()),
        )[0]
        seq.append(n)
    return seq


gen_markov()


In [ ]:
random.choices(["a", "b", "c"], weights=[0.2, 0.9, 0.0], k=10)

## recursive


In [ ]:
rules = {
    "stmt": ["va|opas|expr|pu"],
    "opas": ["=", "<-", "+="],
    "expr": ["va", "num", "f( expr )"],
    "loop": ["for| |va| |in| range(|int|):|\n|    |stmt"],
    "va": ["x", "y", "count"],
    "num": ["42", "0", "3.14"],
    "int": ["1", "2", "3"],
    "f": ["func1", "func2"],
    "brop": ["("],
    "brcl": [")"],
    "op": ["+", "-", "*", "/"],
    "pu": [";", ""],
}


def generate(rule):
    """Recursively expand a rule to generate a sequence."""
    if rule not in rules:  # Base case: terminal token
        return rule

    # Randomly choose a production for this rule
    production = random.choice(rules[rule])

    # Recursively generate for each token in the production
    tokens = []
    for token in production.split("|"):
        tokens.extend(generate(token))

    return tokens


# Example: Start with a "stmt"
tree = generate("loop")
print("".join(tree))